In [1]:
!blastn -task blastn -query data/res/sRNA_DEG.fasta  -subject data/data/miRNAs/mature.fa -outfmt "6 qseqid sseqid qstart qend sstart send score length mismatch gaps gapopen nident pident qlen slen qcovs" > data/res/sRNA_DEG_mirbase.blast.csv

In [2]:
import pandas as pd
data_dir = 'data'
lb = 'iwgsc_10_12_18'
blast_out = data_dir + '/res/sRNA_DEG_mirbase.blast.csv'

In [4]:
deg_file = data_dir + '/res/deg_data.csv'
df_deg = pd.read_csv(deg_file,sep="\t")
print('Res len:',len(df_deg.index))

Res len: 67


In [5]:
df_blast = pd.read_csv(blast_out,sep="\t", header=None)
df_blast.columns = ['qseqid','sseqid','qstart','qend','sstart','send','score','length','mismatch','gaps','gapopen','nident','pident','qlen','slen','qcovs']
print('initial:',len(df_blast.index))

initial: 220


In [6]:
df_blast.head()

,qseqid,sseqid,qstart,qend,sstart,send,score,length,mismatch,gaps,gapopen,nident,pident,qlen,slen,qcovs
0,Cluster_1075,ath-miR5645e,9,21,4,16,26,13,0,0,0,13,100.0,24,20,54
1,Cluster_1075,ath-miR5645f,9,21,4,16,26,13,0,0,0,13,100.0,24,20,54
2,Cluster_1075,ath-miR5645d,9,21,4,16,26,13,0,0,0,13,100.0,24,20,54
3,Cluster_1075,ath-miR5645b,9,21,4,16,26,13,0,0,0,13,100.0,24,20,54
4,Cluster_1075,ath-miR5645a,9,21,4,16,26,13,0,0,0,13,100.0,24,20,54


In [7]:
df_blast = df_blast[df_blast.length >= df_blast.qlen - 4]
print('len -4:',len(df_blast.index))

len -2: 20


In [8]:
df_blast.head()

,qseqid,sseqid,qstart,qend,sstart,send,score,length,mismatch,gaps,gapopen,nident,pident,qlen,slen,qcovs
22,Cluster_6526,tae-miR9776,1,20,2,21,34,20,1,0,0,19,95.0,21,21,95
35,Cluster_8572,tae-miR9652-5p,1,22,1,22,44,22,0,0,0,22,100.0,22,22,100
40,Cluster_10557,tae-miR9776,1,20,2,21,34,20,1,0,0,19,95.0,21,21,95
74,Cluster_22137,ata-miR9672-3p,2,21,2,21,40,20,0,0,0,20,100.0,21,21,95
75,Cluster_22137,tae-miR9672b,2,21,2,21,40,20,0,0,0,20,100.0,21,21,95


In [26]:
df_res = pd.merge(df_deg, df_blast[['qseqid','sseqid']], left_on='Name', right_on='qseqid', how='left')

In [27]:
df_res = df_res.drop(['qseqid'], axis=1)
df_res.rename(columns={'sseqid': 'miRBASE'}, inplace=True)

In [28]:
df_res.to_csv(data_dir + '/res/sRNA_DEG_mirbase.csv', sep='\t', index=None)

In [29]:
len(set(df_res['#Locus'].tolist()))

67